15 mins scalping strategy

In [5]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import numpy as np
import sys
from datetime import datetime

In [2]:

if not mt5.initialize(): 
    print( 'initialization failed')
    quit() 
else: print('initialization success') 



login = 192918303
password = "Demo@1234"
server = "Exness-MT5Trial"

# mt5.login(login, password, server)


if not mt5.login(login, password, server):
    print("Failed to connect to the server")
else:
    print("Connected to the server successfully")
    print(mt5.account_info())


initialization success
Connected to the server successfully
AccountInfo(login=192918303, trade_mode=0, leverage=200, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=10745.11, credit=0.0, profit=0.0, equity=10745.11, margin=0.0, margin_free=10745.11, margin_level=0.0, margin_so_call=60.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DemoBotTester', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')


In [13]:
df= mt5.copy_rates_from_pos('BTCUSDm',mt5.TIMEFRAME_M15,0, 90000 )
df = pd.DataFrame(df)
df['time']= pd.to_datetime(df['time'], unit="s" )
df=dfSPY[df.high!=df.low]
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-06-26 16:45:00,21365.13,21383.18,21304.55,21328.89,1140,1848,0
1,2022-06-26 17:00:00,21329.60,21337.71,21257.46,21290.67,1252,1669,0
2,2022-06-26 17:15:00,21291.82,21308.65,21110.48,21182.94,1389,1542,0
3,2022-06-26 17:30:00,21182.09,21204.39,21139.42,21201.84,1247,1666,0
4,2022-06-26 17:45:00,21204.22,21260.31,21187.78,21239.84,1134,1790,0
...,...,...,...,...,...,...,...,...
89995,2025-01-19 10:00:00,105185.49,105202.83,104614.92,104753.00,4115,2880,0
89996,2025-01-19 10:15:00,104752.40,104936.87,104289.47,104402.99,3800,2880,0
89997,2025-01-19 10:30:00,104402.98,104911.56,104348.88,104865.57,4190,2880,0
89998,2025-01-19 10:45:00,104865.57,105064.39,104697.90,104714.89,3671,2880,0


In [14]:
import pandas_ta as ta
df["EMA"] = ta.ema(df.close, length=100)
df["RSI"] = ta.rsi(df.close, length=3)
#adx - measures the strength o a trend without indicating its direction . 
# returens ADX, DMP, DMN
a=ta.adx(df.high, df.low, df.close, length=7)

df['ADX']=a['ADX_7']
df['ATR']= df.ta.atr()
#help(ta.adx)
a

,ADX_7,DMP_7,DMN_7
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
89995,29.483668,28.413965,23.095955
89996,26.341466,23.731880,27.573776
89997,23.648150,20.334283,23.626150
89998,20.636511,22.430691,21.308056


## EMA Trend Signal

checks if the price is above or below the EMA for a certain number of past candles (backcandles)
The signal is:

3: if the price is both above and below the EMA (trend reversal)

2: if the price is above the EMA (uptrend)

1: if the price is below the EMA (downtrend)

0: otherwise



In [17]:
emasignal = [0]*len(df)
backcandles = 8

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.high[i]>=df.EMA[i]:
            dnt=0 #upt
        if df.low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        # print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMAsignal'] = emasignal

## RSI & ADX Signal
EMAsigma to determine the trend direction 
RSI to meausre market's momentum
- if RSI > 80,overboght teritory, : due for pullback,
- if rsi< 20 , oversold territory: due for a a bounce

ADX to measure the stength of the trend,:
- adx > 30 is a strong trend.

If we're in an uptrend (EMAsignal = 1), the market is overbought (RSI >= 80), and the trend is strong (ADX >= 30), we generate a sell signal (RSIADXSignal = 1). This is because we're expecting the market to pull back from its overbought levels.

If we're in a downtrend (EMAsignal = 2), the market is oversold (RSI <= 20), and the trend is strong (ADX >= 30), we generate a buy signal (RSIADXSignal = 2). This is because we're expecting the market to bounce back from its oversold levels.


In [18]:
RSIADXSignal = [0] * len(df)
for row in range(0, len(df)):
    RSIADXSignal[row] = 0
    if df.EMAsignal[row]==1 and df.RSI[row]>=80 and df.ADX[row]>=30:
        RSIADXSignal[row]=1 # sell signal
    if df.EMAsignal[row]==2 and df.RSI[row]<=20 and df.ADX[row]>=30:
        RSIADXSignal[row]=2

df['RSIADXSignal']=RSIADXSignal

# Candle signal
another trading signal, CandleSignal, based on the previous signal RSIADXSignal and the candlestick patterns in the market.

The previous signal (RSIADXSignal) is used as a filter to determine whether to look for a bearish or bullish engulfing pattern.

If the previous signal is a sell signal (RSIADXSignal == 1),we look for a bearish engulfing pattern, where the current candle's open price is higher than its close price, and the close price is lower than the minimum of the previous candle's close and open prices.

If the previous signal is a buy signal (RSIADXSignal == 2), we looks for a bullish engulfing pattern, where the current candle's open price is lower than its close price, and the close price is higher than the maximum of the previous candle's close and open prices.

If the candlestick pattern confirms the previous signal, a new signal (CandleSignal) is generated.

In [22]:
CandleSignal = [0] * len(df)
for row in range(1, len(df)):
    CandleSignal[row] = 0
    #if (RSIADXSignal[row]==1 or RSIADXSignal[row-1]==1) and (df.Open[row]>df.Close[row]):# and df.Close[row]<df.Close[row-1]):
    #    CandleSignal[row]=1
    #if (RSIADXSignal[row]==2 or RSIADXSignal[row-1]==2) and (df.Open[row]<df.Close[row]):# and df.Close[row]>df.Close[row-1]):
    #    CandleSignal[row]=2
    if RSIADXSignal[row-1]==1 and df.open[row]>df.close[row] and df.close[row]<min(df.close[row-1], df.open[row-1]):
        CandleSignal[row]=1
    if RSIADXSignal[row-1]==2 and df.open[row]<df.close[row] and df.close[row]>max(df.close[row-1], df.open[row-1]):
        CandleSignal[row]=2

df['TotSignal']=CandleSignal

# Visualizing Signals

In [24]:
import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['high']+1e-4
    elif x['TotSignal']==2:
        return x['low']-1e-4
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[0:1000]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='red', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [26]:
df.iloc[667:672]

,time,open,high,low,close,tick_volume,spread,real_volume,EMA,RSI,ADX,ATR,EMAsignal,RSIADXSignal,TotSignal,pointpos
667,2022-07-03 15:30:00,19055.30,19063.13,19012.64,19049.89,896,1684,0,19128.117368,60.693964,21.910660,55.157911,1,0,0,NaN
668,2022-07-03 15:45:00,19049.07,19051.28,19005.00,19014.19,894,1694,0,19125.861380,21.988097,19.501038,54.523775,1,0,0,NaN
669,2022-07-03 16:00:00,19015.29,19030.42,18991.01,19024.06,910,1648,0,19123.845511,38.304517,19.130061,53.444219,1,0,0,NaN
670,2022-07-03 16:15:00,19023.88,19076.03,19018.89,19069.69,897,1641,0,19122.773125,74.822307,19.361200,53.708204,1,0,0,NaN
671,2022-07-03 16:30:00,19070.48,19171.74,19061.12,19115.74,974,1697,0,19122.633855,86.720822,24.550765,57.773332,0,0,0,NaN


# Stoploss from price
Stop-Loss (SL) signal based on the previous trading signal (TotSignal).

It sets a parameter SLbackcandles to 2, which determines how many previous candles to consider when calculating the Stop-Loss.

It iterates through the dataframe, starting from the SLbackcandles-th row (since it needs to look back SLbackcandles rows).

For each row, it checks the value of TotSignal. If it's 1 (sell signal), it calculates the maximum high price of the previous SLbackcandles rows and sets it as the Stop-Loss value.

If TotSignal is 2 (buy signal), it calculates the minimum low price of the previous SLbackcandles rows and sets it as the Stop-Loss value.

The resulting Stop-Loss values are stored in the SLSignal array and added to the dataframe as a new column.

In [27]:
SLSignal = [0] * len(df)
SLbackcandles = 2
for row in range(SLbackcandles, len(df)):
    mi=1e10
    ma=-1e10
    if df.TotSignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,df.high[i])
        SLSignal[row]=ma
    if df.TotSignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,df.low[i])
        SLSignal[row]=mi
        
df['SLSignal']=SLSignal

In [ ]:
# Backtesting strategy

In [35]:
dfpl = df[:]
def SIGNAL():
    return dfpl.TotSignal
dfpl

,time,open,high,low,close,tick_volume,spread,real_volume,EMA,RSI,ADX,ATR,EMAsignal,RSIADXSignal,TotSignal,pointpos,SLSignal
0,2022-06-26 16:45:00,21365.13,21383.18,21304.55,21328.89,1140,1848,0,NaN,NaN,NaN,NaN,0,0,0,NaN,0.0
1,2022-06-26 17:00:00,21329.60,21337.71,21257.46,21290.67,1252,1669,0,NaN,NaN,NaN,NaN,0,0,0,NaN,0.0
2,2022-06-26 17:15:00,21291.82,21308.65,21110.48,21182.94,1389,1542,0,NaN,NaN,NaN,NaN,0,0,0,NaN,0.0
3,2022-06-26 17:30:00,21182.09,21204.39,21139.42,21201.84,1247,1666,0,NaN,17.547660,NaN,NaN,0,0,0,NaN,0.0
4,2022-06-26 17:45:00,21204.22,21260.31,21187.78,21239.84,1134,1790,0,NaN,46.081923,NaN,NaN,0,0,0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,2025-01-19 10:00:00,105185.49,105202.83,104614.92,104753.00,4115,2880,0,104297.660882,54.726193,29.483668,453.393844,0,0,0,NaN,0.0
89996,2025-01-19 10:15:00,104752.40,104936.87,104289.47,104402.99,3800,2880,0,104299.746607,38.487423,26.341466,467.251426,0,0,0,NaN,0.0
89997,2025-01-19 10:30:00,104402.98,104911.56,104348.88,104865.57,4190,2880,0,104310.951031,61.270006,23.648150,474.067753,0,0,0,NaN,0.0
89998,2025-01-19 10:45:00,104865.57,105064.39,104697.90,104714.89,3671,2880,0,104318.949823,51.881224,20.636511,466.383628,0,0,0,NaN,0.0


In [42]:
# rename columns o fit backtesting parameters
dfpl = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
dfpl.set_index("time", inplace=True)


In [39]:
from backtesting import Strategy
from backtesting import Backtest
class MyStrat(Strategy):
    initsize = 0.03
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.3*self.data.ATR[-1]
        TPSLRatio = 1.3

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/50, commission=.00)
stat = bt.run()
stat

C:\Users\derne\AppData\Local\Temp\ipykernel_84736\2599756411.py:25: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).



Start                     2022-06-26 16:45:00
End                       2025-01-19 11:00:00
Duration                    937 days 18:15:00
Exposure Time [%]                    6.045556
Equity Final [$]                 89178.716368
Equity Peak [$]                 103363.338309
Return [%]                         -10.821284
Buy & Hold Return [%]              392.185482
Return (Ann.) [%]                   -4.354186
Volatility (Ann.) [%]                10.19357
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -20.3974
Avg. Drawdown [%]                   -2.682467
Max. Drawdown Duration      915 days 12:45:00
Avg. Drawdown Duration       84 days 09:35:00
# Trades                                  540
Win Rate [%]                        43.148148
Best Trade [%]                       3.106194
Worst Trade [%]                     -2.014452
Avg. Trade [%]                    

In [40]:
bt.plot(show_legend=False)

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:122: UserWarning:

Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:126: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:128: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:136: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\derne\OneDrive - The Pe

ValueError: Length of values (2) does not match length of index (1)

In [44]:

from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.02
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/50, commission=.00)
stat = bt.run()
stat

C:\Users\derne\AppData\Local\Temp\ipykernel_84736\3409065464.py:25: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).



Start                     2022-06-26 16:45:00
End                       2025-01-19 11:00:00
Duration                    937 days 18:15:00
Exposure Time [%]                    3.603333
Equity Final [$]                    100931.63
Equity Peak [$]                     108368.41
Return [%]                            0.93163
Buy & Hold Return [%]              392.185482
Return (Ann.) [%]                    0.361109
Volatility (Ann.) [%]                5.595107
Sharpe Ratio                          0.06454
Sortino Ratio                        0.092112
Calmar Ratio                         0.033225
Max. Drawdown [%]                  -10.868619
Avg. Drawdown [%]                   -0.881412
Max. Drawdown Duration      801 days 22:45:00
Avg. Drawdown Duration       31 days 06:24:00
# Trades                                  545
Win Rate [%]                        50.642202
Best Trade [%]                       2.113715
Worst Trade [%]                     -1.868083
Avg. Trade [%]                    

In [45]:
bt.plot(show_legend=False)

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:122: UserWarning:

Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:126: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:128: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\env\Lib\site-packages\backtesting\_plotting.py:136: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\derne\OneDrive - The Pe

ValueError: Length of values (2) does not match length of index (1)